# Codon Optimization Dataset Creation

To begin, let's create a 50/50 class split of optimized and non-optimized of sequences. To do this, we'll load up all our RefSeq CDSs and then randomly sort them into either class.

In [1]:
version = "0.1.1"

In [2]:
from random import shuffle, seed, choice
from Bio import SeqIO
seed(42) # for reproducibility

# load the sequences and shuffle them randomly
CDS = list(SeqIO.parse("inserts/all_refseq_prokaryote_CDS.fasta", "fasta"))
CDS = list(set([str(x.seq) for x in CDS])) # to remove duplicates
shuffle(CDS)

# split them 50/50 and remove the first codon, which will always translate to M
optimized, natural = CDS[:int(len(CDS)/2)], CDS[int(len(CDS)/2):]
optimized, natural = [seq[3:] for seq in optimized], [seq[3:] for seq in natural] # remove the first codon, which is always translated as an M
len(optimized), len(natural)

(201278, 201278)

## Target Data Collection

Before we can do our optimization, we'll need to collect the data on our targets, which will be highly expressed and overall genome CUB.

### Highly Expressed Genes (HEGs) Targets
The HEGs are stored as a list of FASTA files in the targets/heg directory. We are going to iterate over them and calculate their codon usage bias.

In [3]:
import os
import freqgen

heg_cub = []
heg_names = []

for heg in os.listdir("targets/heg"):
    
    # only deal with fasta files
    if not heg.endswith(".fasta"):
        continue
        
    # calculate the HEGs' CUB
    seq = [x.seq for x in SeqIO.parse(f'targets/heg/{heg}', "fasta")]
    try:
        heg_cub.append(freqgen.codon_frequencies(seq))
        heg_names.append(heg)
    except ValueError: # ignore bad codons
        pass

### Genome Targets
In addition to targeting various species' HEGs, we will also target overall genome CUB. To do so, we are using the data from [this paper](https://doi.org/10.1186/s12859-017-1793-7). It is stored as a large CSV file, so we'll use pandas to read it in and convert it to list of dicts of CUB.

In [4]:
import pandas as pd
df = pd.read_csv("targets/o563612-refseq_species.tsv", sep="\t")

/Users/BenjaminLee/Desktop/Python/Research/condo/env/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/BenjaminLee/Desktop/Python/Research/condo/env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
from tqdm import trange

genome_cub = [] # a list of the each species' CUB
genome_names = [] # the name of each species

for i in trange(len(df)):
    genome_cub.append(dict(df.iloc[i][12:])) # the raw codon counts
    genome_names.append(df.iloc[i][3]) # species name

100%|██████████| 114111/114111 [03:37<00:00, 523.46it/s]


In [6]:
# how many targets are there?
targets = genome_cub + heg_cub
len(targets)

114307

## Optimize the sequences

Ok, now we have all our targets and inserts. We'll iterate over the CDSs we've selected for optimization and optimize them using one of the methods towards one of the targets.

In [7]:
import Bio.Data.CodonTable as ct

def _synonymous_codons(genetic_code_dict):

    # invert the genetic code dictionary to map each amino acid to its codons
    codons_for_amino_acid = {}
    for codon, amino_acid in genetic_code_dict.items():
        codons_for_amino_acid[amino_acid] = codons_for_amino_acid.get(amino_acid, [])
        codons_for_amino_acid[amino_acid].append(codon)

    # create dictionary of synonymous codons
    # Example: {'CTT': ['CTT', 'CTG', 'CTA', 'CTC', 'TTA', 'TTG'], 'ATG': ['ATG']...}
    return {codon: codons_for_amino_acid[genetic_code_dict[codon]] for codon in genetic_code_dict.keys()}


synonymous_codons = {k: _synonymous_codons(v.forward_table) for k, v in ct.unambiguous_dna_by_id.items()}


In [8]:
def absolute_to_relative(target, trans_table):
    relative = {}
    for i in synonymous_codons[trans_table].keys():
        try:
            relative[i] = target[i] / sum((target[codon] for codon in synonymous_codons[trans_table][i]))
        except ZeroDivisionError:
            relative[i] = 1 / len(synonymous_codons[trans_table][i]) # if an amino acid is never used in the reference set, then all its codons are used equally
    return relative

In [9]:
from Bio.Seq import Seq
import numpy as np
np.random.seed(42)

def CAI_max(seq, absolute, trans_table):
    relative = absolute_to_relative(absolute, trans_table)
    optimized_seq = ""
    for codon in [seq[i:i + 3] for i in range(0, len(seq), 3)]:
        optimized_seq += max([(c, relative[c]) for c in synonymous_codons[trans_table][codon]], key=lambda x: x[1])[0] # chose the most used synonym
    return optimized_seq

def multinomial(seq, absolute, trans_table):
    relative = absolute_to_relative(absolute, trans_table)
    optimized_seq = ""
    for codon in [seq[i:i + 3] for i in range(0, len(seq), 3)]:
        optimized_seq += np.random.choice(synonymous_codons[trans_table][codon], p=[relative[c] for c in synonymous_codons[trans_table][codon]]) # pick a codon with the frequencies of the target
    return optimized_seq

In [10]:
def optimize(seq):
    target_method = np.random.choice(["cai_max", "multinomial"])#, "freqgen"])
    
    targets = [heg_cub, genome_cub]
    target_type = np.random.choice(targets)
    target_idx = np.random.randint(len(target_type))
    target = target_type[target_idx]
    target_name = [heg_names, genome_names][targets.index(target_type)][target_idx]
    
    if target_type == genome_cub:
        trans_table = df.iloc[target_idx]["Translation Table"]
    else:
        trans_table = 11
    
    if target_method == "cai_max":
        result = CAI_max(seq, target, trans_table)
    elif target_method == "multinomial":
        result = multinomial(seq, target, trans_table)
    return dict(sequence=result,
                method=target_method, 
                target_type=["heg", "genome"][targets.index(target_type)], 
                target_name=target_name,
                trans_table=trans_table,
                optimized=1)

In [11]:
from tqdm import tqdm

results = []
for sequence in tqdm(optimized):
    try:
        results.append(optimize(sequence))
    except KeyError:
        pass

100%|██████████| 201278/201278 [42:43<00:00, 78.51it/s]


## Output a CSV

In [12]:
for sequence in tqdm(natural):
    results.append(dict(sequence=sequence,
                        optimized=0,
                        method=None,
                        target_type=None,
                        target_name=None,
                        trans_table=11,))

100%|██████████| 201278/201278 [00:01<00:00, 137391.25it/s]


In [13]:
columns = ["sequence", "optimized", "method", "trans_table", "target_type", "target_name"]
pd.DataFrame(results, columns=columns).to_csv(f"data/condo-{version}.csv", index=False)